## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
#### <center>ФБ-25 Савченко Єлизавета</center>

In [1]:
!pip install pandas --break-system-packages


Defaulting to user installation because normal site-packages is not writeable


Імпортуємо бібліотеки

In [2]:
import datetime
import requests
import urllib.request
import pandas as pd
import os

Скачуємо індекси для всіх областей. Додаємо в назву дату та час скачування

In [3]:
if os.path.exists('vhi'):
    os.rename('vhi', 'vhi_backup_' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
os.mkdir('vhi')

for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Download complete")
        break

    date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    vhi_url = urllib.request.urlopen(url)
    file_name = f'vhi/vhi_{ids}_{date_now}.csv'
    out = open(file_name, 'wb')
    out.write(vhi_url.read())
    out.close()
    print(f"VHI from id {ids} was downloaded at {date_now}")

VHI from id 1 was downloaded at 2024-04-09_22-59-59
VHI from id 2 was downloaded at 2024-04-09_23-00-01
VHI from id 3 was downloaded at 2024-04-09_23-00-03
VHI from id 4 was downloaded at 2024-04-09_23-00-05
VHI from id 5 was downloaded at 2024-04-09_23-00-08
VHI from id 6 was downloaded at 2024-04-09_23-00-10
VHI from id 7 was downloaded at 2024-04-09_23-00-12
VHI from id 8 was downloaded at 2024-04-09_23-00-14
VHI from id 9 was downloaded at 2024-04-09_23-00-16
VHI from id 10 was downloaded at 2024-04-09_23-00-18
VHI from id 11 was downloaded at 2024-04-09_23-00-20
VHI from id 12 was downloaded at 2024-04-09_23-00-23
VHI from id 13 was downloaded at 2024-04-09_23-00-24
VHI from id 14 was downloaded at 2024-04-09_23-00-26
VHI from id 15 was downloaded at 2024-04-09_23-00-28
VHI from id 16 was downloaded at 2024-04-09_23-00-30
VHI from id 17 was downloaded at 2024-04-09_23-00-32
VHI from id 18 was downloaded at 2024-04-09_23-00-35
VHI from id 19 was downloaded at 2024-04-09_23-00-37
VH

Даємо імена стовбцям та зчитуємо файли у фрейм

In [4]:
folder = 'vhi'
files = os.listdir(folder)
df_all = pd.DataFrame()

for file_name in files:
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)
    df['ProvinceId'] = int(file_name.split("_")[1])
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df = df.drop(columns=['empty'])
    df_all = pd.concat([df_all, df]).drop_duplicates().reset_index(drop=True)

df_all = df_all.dropna(axis=0, how='any')
print(df_all, '\n')


       Year  Week    SMN     SMT    VCI    TCI    VHI  ProvinceId
0      1982   2.0  0.047  258.77  42.75  49.09  45.92          22
1      1982   3.0  0.045  260.25  43.44  43.56  43.50          22
2      1982   4.0  0.040  261.37  37.43  40.81  39.12          22
3      1982   5.0  0.037  262.61  30.04  41.17  35.60          22
4      1982   6.0  0.035  263.47  24.65  42.49  33.57          22
...     ...   ...    ...     ...    ...    ...    ...         ...
57990  2024  10.0  0.129  283.12  77.82   4.43  41.12           6
57991  2024  11.0  0.141  285.05  75.25   5.68  40.46           6
57992  2024  12.0  0.153  287.23  73.17   6.86  40.02           6
57993  2024  13.0  0.164  289.63  70.63   8.77  39.70           6
57994  2024  14.0  0.174  291.18  66.23  10.76  38.49           6

[57969 rows x 8 columns] 



Змінюємо індекси областей

In [5]:
ProvinceId_replacements = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["ProvinceId"] = df_all["ProvinceId"].replace(ProvinceId_replacements)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  ProvinceId
0      1982   2.0  0.047  258.77  42.75  49.09  45.92          18
1      1982   3.0  0.045  260.25  43.44  43.56  43.50          18
2      1982   4.0  0.040  261.37  37.43  40.81  39.12          18
3      1982   5.0  0.037  262.61  30.04  41.17  35.60          18
4      1982   6.0  0.035  263.47  24.65  42.49  33.57          18
...     ...   ...    ...     ...    ...    ...    ...         ...
57990  2024  10.0  0.129  283.12  77.82   4.43  41.12           4
57991  2024  11.0  0.141  285.05  75.25   5.68  40.46           4
57992  2024  12.0  0.153  287.23  73.17   6.86  40.02           4
57993  2024  13.0  0.164  289.63  70.63   8.77  39.70           4
57994  2024  14.0  0.174  291.18  66.23  10.76  38.49           4

[57969 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [8]:
def vhi(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI']

def vhi_min(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI'].min()

def vhi_max(ProvinceId, year):
    filtered_df = df_all[(df_all["ProvinceId"] == ProvinceId) & (df_all["Year"] == year)]
    return filtered_df['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)


In [9]:
vhi(6, 2005)

14034    47.42
14035    51.20
14036    53.68
14037    54.35
14038    54.66
14039    56.10
14040    56.81
14041    56.06
14042    56.27
14043    56.51
14044    56.42
14045    56.02
14046    54.84
14047    52.12
14048    49.52
14049    48.44
14050    53.62
14051    52.32
14052    52.21
14053    51.77
14054    53.50
14055    54.49
14056    54.25
14057    55.50
14058    57.29
14059    57.93
14060    58.44
14061    65.13
14062    59.35
14063    61.40
14064    63.51
14065    64.39
14066    66.13
14067    65.04
14068    60.86
14069    57.68
14070    56.59
14071    54.98
14072    53.23
14073    52.48
14074    55.20
14075    57.22
14076    57.66
14077    58.95
14078    56.50
14079    54.77
14080    54.48
14081    53.22
14082    50.36
14083    47.41
14084    45.59
Name: VHI, dtype: float64

In [10]:
vhi_min(6, 2005)


45.59

In [11]:
vhi_max(6, 2005)

66.13

Ряд VHI за вказаний діапазон років для вказаних областей

In [12]:
def vhi_range(year_min, year_max, ProvinceIds):
    if not isinstance(ProvinceIds, list) or not ProvinceIds:
        print('Empty or not a list')
        return
    filtered_df = df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['ProvinceId'].isin(ProvinceIds))]
    return filtered_df[['Year', 'VHI', 'ProvinceId']]


In [13]:
vhi_range(2000, 2004, [5, 2, 3])

,Year,VHI,ProvinceId
11645,2000,39.02,3
11646,2000,42.24,3
11647,2000,45.66,3
11648,2000,47.38,3
11649,2000,48.96,3
...,...,...,...
44101,2004,55.14,2
44102,2004,52.90,2
44103,2004,50.57,2
44104,2004,50.45,2


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [14]:
def extreme_droughts(percent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['ProvinceId'].nunique()
    result = grouped[grouped > (25 * percent / 100)].reset_index()
    return result

In [15]:
extreme_droughts(5)

,Year,ProvinceId
0,1986,2
1,1993,2
2,2000,6
3,2007,5


І так само для помірних посух

In [16]:
def moderate_droughts(percent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['ProvinceId'].nunique()
    count_areas = df_all['ProvinceId'].nunique()
    result = grouped[grouped > (count_areas * percent / 100)].reset_index()
    return result

In [17]:
moderate_droughts(10)

,Year,ProvinceId
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
